## Fuzzy Deduplication Workflow

This notebook shows how to:
1. Load config from `config.yml`.
2. Load and unify Dataset1 + Dataset2 (in Dataset1’s format).
3. Block and fuzzy-match records.
4. Merge duplicates.
5. Save final combined data.

In [1]:
import pandas as pd
from src.fuzzy_helpers import (
    load_config,
    load_dataset1,
    load_dataset2_and_map,
    combine_dataframes,
    block_data,
    build_comparison_features,
    classify_duplicates,
    merge_duplicate_pairs
)

### 1. Load Configuration

Our `config.yml` file contains:
- dataset1_path
- dataset2_path
- output_path
- dataset1_columns
- dataset2_to_dataset1
- blocking_keys
- fuzzy_keys
- similarity_threshold

In [2]:
config_path = "config.yml"  # Adjust if needed
config = load_config(config_path)

dataset1_path = config["dataset1_path"]
dataset2_path = config["dataset2_path"]
output_path = config["output_path"]

dataset1_columns = config["dataset1_columns"]
dataset2_to_dataset1 = config["dataset2_to_dataset1"]
blocking_keys = config["blocking_keys"]
fuzzy_keys = config["fuzzy_keys"]
similarity_threshold = float(config["similarity_threshold"])

### 2. Load Datasets

Dataset1 is presumably already in the final schema, but we ensure all columns exist.  
Dataset2 is mapped to that same schema using the provided dictionary.


In [3]:
df1 = load_dataset1(dataset1_path, dataset1_columns)
df2_mapped = load_dataset2_and_map(dataset2_path, dataset1_columns, dataset2_to_dataset1)

In [4]:
df1

,Region,Market,Country,Opportunity Name,Customer Name,Submission Due Date,Supply Start Date,Opportunity Expiration Date,Molecule,Local Product Description,...,Net Revenue Currency,Net Revenue,Net Revenue (converted) Currency,Net Revenue (converted),Outcome,Volume Won (in packs),Outcome Reason,Winning Competitor,Winning Competitor Price Currency,Winning Competitor Price
0,APAC,Oncology,Japan,APAC-ONC-1,Tokyo University Hospital,11/15/23,1/1/24,1/1/25,Trastuzumab,TRAZIMERA (Trastuzumab) 150mg,...,JPY,2000000,USD,17500,Pending,0,NaN,Mylan,JPY,3800.0
1,APAC,Oncology,Japan,APAC-ONC-2,Tokyo Univ Hospital,11/15/23,1/1/24,1/1/25,Trastuzumab,Trazimera (TRASTUZUMAB) 150 mg,...,JPY,2025000,USD,18225,Pending,0,NaN,Mylan,JPY,3850.0
2,APAC,Oncology,China,APAC-ONC-3,Beijing Cancer Center,12/1/23,2/1/24,2/1/25,Bevacizumab,AVASTIN (Bevacizumab) 100mg,...,CNY,60000,USD,8400,Won,300,NaN,NaN,CNY,NaN
3,APAC,Oncology,China,APAC-ONC-4,Beijing Cancer Ctr,12/1/23,2/1/24,2/1/25,Bevacizumab,Avastin (Bevacizumab) 100 mg,...,CNY,64105,USD,9285,Won,305,NaN,NaN,CNY,NaN
4,APAC,Oncology,South Korea,APAC-ONC-5,Seoul National Hosp,10/10/23,3/1/24,3/1/26,Trastuzumab,TRAZIMERA (Trastuzumab) 420mg,...,KRW,4000000,USD,3360,Lost,0,Price too high,Sandoz,KRW,4800.0
5,APAC,Biosimilars,Japan,APAC-BIO-1,Osaka Clinic,9/20/23,1/15/24,1/14/25,Etanercept,ENBREL (Etanercept) 50mg,...,JPY,1200000,USD,10400,Pending,0,NaN,Biocon,JPY,2900.0
6,APAC,Biosimilars,Japan,APAC-BIO-2,Osaka Clinc,9/20/23,1/15/24,1/14/25,Etanercept,Enbrel (ETANERCEPT) 50 mg,...,JPY,1250000,USD,11250,Pending,0,NaN,Biocon,JPY,2950.0
7,APAC,Biosimilars,China,APAC-BIO-3,Shanghai Research Hosp,10/1/23,2/10/24,2/9/25,Adalimumab,Humira (Adalimumab) 40mg,...,CNY,350000,USD,50000,Won,1000,NaN,Biocon,CNY,320.0
8,APAC,Biosimilars,China,APAC-BIO-4,Shanghai Rsrch Hospital,10/1/23,2/10/24,2/9/25,Adalimumab,HUMIRA (Adalimumab) 40 mg,...,CNY,361800,USD,51960,Won,1005,NaN,Biocon,CNY,325.0
9,APAC,Biosimilars,South Korea,APAC-BIO-5,Seoul Clinic,7/15/23,3/20/24,3/19/25,Bevacizumab,MVASI (Bevacizumab) 100mg,...,KRW,2000000,USD,1750,Lost,0,Lack of funds,Sandoz,KRW,3800.0


In [5]:
df2_mapped

,Region,Market,Country,Opportunity Name,Customer Name,Submission Due Date,Supply Start Date,Opportunity Expiration Date,Molecule,Local Product Description,...,Net Revenue Currency,Net Revenue,Net Revenue (converted) Currency,Net Revenue (converted),Outcome,Volume Won (in packs),Outcome Reason,Winning Competitor,Winning Competitor Price Currency,Winning Competitor Price
0,APAC,Oncology,Japan,APAC-ONC-1 alt,Tokyo University Hosp,11/15/23,1/1/24,1/1/25,Trastuzumab,Trazimera (Trastuzumab) 150 mg,...,JPY,2050000,USD,18000.0,Pending,0,NaN,Mylan,JPY,3900.0
1,APAC,Oncology,Japan,APAC-ONC-2,Tokyo University Hospital,11/15/23,1/1/24,1/1/25,Trastuzumab,TRAZIMERA (TRASTUZUMAB) 150 mg,...,JPY,2030000,USD,18270.0,Pending,0,NaN,Mylan,JPY,3860.0
2,APAC,Oncology,China,APAC-ONC-3,Beijing Cancer Center,12/1/23,2/1/24,2/1/25,Bevacizumab,Avastin (Bevacizumab) 100 mg,...,CNY,61500,USD,8700.0,Won,300,NaN,NaN,CNY,NaN
3,APAC,Oncology,China,APAC-ONC-4 alt,Beijing Cancer Ctr,12/1/23,2/1/24,2/1/25,Bevacizumab,AVASTIN (Bevacizumab) 100mg,...,CNY,65575,USD,9375.0,Won,305,NaN,NaN,CNY,NaN
4,APAC,Oncology,South Korea,APAC-ONC-5,Seoul National Hospital,10/10/23,3/1/24,3/1/26,Trastuzumab,TRAZIMERA (Trastuzumab) 420 mg,...,KRW,4100000,USD,3444.0,Lost,0,Price too high,Sandoz,KRW,4850.0
5,APAC,Biosimilars,Japan,APAC-BIO-1 alt,Osaka Clinic,9/20/23,1/15/24,1/14/25,Etanercept,Enbrel (Etanercept) 50 mg,...,JPY,1210000,USD,10800.0,Pending,0,NaN,Biocon,JPY,2910.0
6,APAC,Biosimilars,Japan,APAC-BIO-2,Osaka Clinc,9/20/23,1/15/24,1/14/25,Etanercept,ENBREL (ETANERCEPT) 50 mg,...,JPY,1270000,USD,11380.0,Pending,0,NaN,Biocon,JPY,2960.0
7,APAC,Biosimilars,China,APAC-BIO-3,Shanghai Research Hospital,10/1/23,2/10/24,2/9/25,Adalimumab,Humira (ADALIMUMAB) 40 mg,...,CNY,355000,USD,50500.0,Won,1000,NaN,Biocon,CNY,330.0
8,APAC,Biosimilars,China,APAC-BIO-4 alt,Shanghai Rsrch Hospital,10/1/23,2/10/24,2/9/25,Adalimumab,Humira (Adalimumab) 40mg,...,CNY,371850,USD,53390.0,Won,1005,NaN,Biocon,CNY,335.0
9,APAC,Biosimilars,South Korea,APAC-BIO-5,Seoul Clinic,7/15/23,3/20/24,3/19/25,Bevacizumab,Mvasi (Bevacizumab) 100mg,...,KRW,2025000,USD,1780.0,Lost,0,Lack of funds,Sandoz,KRW,3850.0


### 3. Combine

In [6]:
combined_df = combine_dataframes(df1, df2_mapped)

**Optional**: Some quick cleanup, e.g., filling NaNs with empty strings in text columns.


In [7]:
# for col in combined_df.columns:
#     if combined_df[col].dtype == object:
#         combined_df[col] = combined_df[col].fillna("")

### 4. Block Data

We create candidate pairs only for rows that share the same values in `blocking_keys`.

In [8]:
candidate_pairs = block_data(combined_df, blocking_keys)

### 5. Build Fuzzy Comparison Features

Using jaro-winkler on the `fuzzy_keys` columns.

In [9]:
features_df = build_comparison_features(combined_df, candidate_pairs, fuzzy_keys)
features_df.head()

0         1         2
0 1   0.96  0.920421  0.755813
  5   0.92  0.468889  0.560750
  6   0.88  0.418182  0.538062
  10  0.88  0.653401  0.618147
  11  0.84  0.653401  0.502246

### 6. Classify Duplicates

If the average similarity across these fields is >= `similarity_threshold`, 
they're flagged as duplicates.

In [10]:
duplicates_idx = classify_duplicates(features_df, similarity_threshold)
print(f"Found {len(duplicates_idx)} pairs classified as duplicates.")

Found 66 pairs classified as duplicates.


### 7. Merge Duplicate Pairs

We'll keep the earliest row from each connected group of duplicates.

In [11]:
deduped_df = merge_duplicate_pairs(combined_df, duplicates_idx)

In [12]:
deduped_df

,Region,Market,Country,Opportunity Name,Customer Name,Submission Due Date,Supply Start Date,Opportunity Expiration Date,Molecule,Local Product Description,...,Net Revenue Currency,Net Revenue,Net Revenue (converted) Currency,Net Revenue (converted),Outcome,Volume Won (in packs),Outcome Reason,Winning Competitor,Winning Competitor Price Currency,Winning Competitor Price
0,APAC,Oncology,Japan,APAC-ONC-1,Tokyo University Hospital,11/15/23,1/1/24,1/1/25,Trastuzumab,TRAZIMERA (Trastuzumab) 150mg,...,JPY,2000000,USD,17500.0,Pending,0,NaN,Mylan,JPY,3800.0
1,APAC,Oncology,Japan,APAC-ONC-2,Tokyo Univ Hospital,11/15/23,1/1/24,1/1/25,Trastuzumab,Trazimera (TRASTUZUMAB) 150 mg,...,JPY,2025000,USD,18225.0,Pending,0,NaN,Mylan,JPY,3850.0
2,APAC,Oncology,China,APAC-ONC-3,Beijing Cancer Center,12/1/23,2/1/24,2/1/25,Bevacizumab,AVASTIN (Bevacizumab) 100mg,...,CNY,60000,USD,8400.0,Won,300,NaN,NaN,CNY,NaN
3,APAC,Oncology,China,APAC-ONC-4,Beijing Cancer Ctr,12/1/23,2/1/24,2/1/25,Bevacizumab,Avastin (Bevacizumab) 100 mg,...,CNY,64105,USD,9285.0,Won,305,NaN,NaN,CNY,NaN
4,APAC,Oncology,South Korea,APAC-ONC-5,Seoul National Hosp,10/10/23,3/1/24,3/1/26,Trastuzumab,TRAZIMERA (Trastuzumab) 420mg,...,KRW,4000000,USD,3360.0,Lost,0,Price too high,Sandoz,KRW,4800.0
5,APAC,Biosimilars,Japan,APAC-BIO-1,Osaka Clinic,9/20/23,1/15/24,1/14/25,Etanercept,ENBREL (Etanercept) 50mg,...,JPY,1200000,USD,10400.0,Pending,0,NaN,Biocon,JPY,2900.0
6,APAC,Biosimilars,Japan,APAC-BIO-2,Osaka Clinc,9/20/23,1/15/24,1/14/25,Etanercept,Enbrel (ETANERCEPT) 50 mg,...,JPY,1250000,USD,11250.0,Pending,0,NaN,Biocon,JPY,2950.0
7,APAC,Biosimilars,China,APAC-BIO-3,Shanghai Research Hosp,10/1/23,2/10/24,2/9/25,Adalimumab,Humira (Adalimumab) 40mg,...,CNY,350000,USD,50000.0,Won,1000,NaN,Biocon,CNY,320.0
8,APAC,Biosimilars,China,APAC-BIO-4,Shanghai Rsrch Hospital,10/1/23,2/10/24,2/9/25,Adalimumab,HUMIRA (Adalimumab) 40 mg,...,CNY,361800,USD,51960.0,Won,1005,NaN,Biocon,CNY,325.0
9,APAC,Biosimilars,South Korea,APAC-BIO-5,Seoul Clinic,7/15/23,3/20/24,3/19/25,Bevacizumab,MVASI (Bevacizumab) 100mg,...,KRW,2000000,USD,1750.0,Lost,0,Lack of funds,Sandoz,KRW,3800.0


In [13]:
print(f"Deduplicated dataframe has {len(deduped_df)} rows.")

Deduplicated dataframe has 59 rows.


### 8. Save Final

In [14]:
deduped_df.to_csv(output_path, index=False)
print(f"De-duplicated dataset saved to: {output_path}")

De-duplicated dataset saved to: data/combined_deduplicated.csv
